# 필요 라이브러리 설치

In [ ]:
!pip install -U accelerate huggingface-hub==0.23.0 transformers==4.40.1 datasets==2.19.0

# 라이브러리

In [ ]:
import os
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    GPT2LMHeadModel,
    AutoTokenizer,
    pipeline,
    logging
    )
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from datasets import load_dataset

import re
import numpy as np
import pandas as pd


In [2]:
# 허깅페이스에 로그인하기
import huggingface_hub
from huggingface_hub import Repository

In [ ]:
# 허깅페이스에 로그인하기.
token = "your_token"
huggingface_hub.login(token=token)

# 데이터 전처리

In [ ]:
# 데이터셋 불러오기
data = "huggingface_dataset_path"
dataset = load_dataset(data, split="train")

# 토크나이저와 모델 불러오기
model_name = "skt/kogpt2-base-v2"
U_TKN = '<usr>'
S_TKN = '<sys>'
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [6]:
# 데이터셋 클래스 정의
class ChatbotDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_len=70):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # 데이터셋에서 특정 인덱스의 입력과 출력 문장 가져오기
        sample = self.dataset[idx]

        input_text = sample.split('[INST]손님:')[1].split('[/INST]')[0]
        output_text = sample.split('[/INST]직원:')[1].split('</s>')[0]

        # 입력 문장을 토큰화하고 패딩 처리
        inputs = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # 출력 문장을 토큰화하고 패딩 처리
        outputs = self.tokenizer.encode(
            output_text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': np.array(outputs)
        }

In [7]:
# 데이터셋 생성
data = ChatbotDataset(dataset['text'], tokenizer)

# 데이터 학습

In [8]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',          # 출력 디렉토리
    overwrite_output_dir=True,
    num_train_epochs=40,                  # 총 학습 에폭 수
    per_device_train_batch_size=16,   # 각 장치 당 배치 크기
    save_steps=500,
    logging_steps=200,
    save_total_limit=2,
)

# Trainer 설정
trainer = Trainer(
    model=model,                      # 학습할 모델
    args=training_args,               # TrainingArguments 전달
    train_dataset=data,               # 학습용 데이터셋
    tokenizer=tokenizer               # 토크나이저
)

In [ ]:
trainer.train()

# 올리기 전 확인하기

In [ ]:
prompt = "예약 가능한가요?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=50)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])


input_text = "예약 가능한가요?"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

output = model.generate(input_ids, max_length=50, num_return_sequences=1, temperature=1.0, pad_token_id=tokenizer.eos_token_id)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

# 모델 허깅페이스에 올리기

In [12]:
# 학습한 모델 hugging face 내 저장한 모델경로 설정
save_model = "huggingface_model_save_path"

In [ ]:
# 허깅페이스에 바로 모델 올리기.
model.push_to_hub(
			save_model,
			use_temp_dir=True,
			use_auth_token=token
)
tokenizer.push_to_hub(
			save_model,
			use_temp_dir=True,
			use_auth_token=token
)